# Simulation code

Try the below situations. Both of them are observable heterogeneity. And not high dimensional case.

1. $y_i = x_i^{'} \beta + \gamma d_i + \epsilon_i$
3. $y_i = x_i^{'} \beta + d_i x_i^{'} \gamma + \epsilon_i$

$x_i = (1,\ z_i)^{'}$ where $z_i \sim N(0,1)$.

In [2]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

## specification 1
$\beta = (1,-1)^{'}, \gamma = 1$

In [3]:
# data generation
sample_size = 100000
half_size = int(sample_size/2)
z = np.random.normal(size = (sample_size,1))
d1 = np.ones((half_size,1))
d0 = np.zeros((half_size,1))
d = np.vstack((d0,d1))
const = np.ones((sample_size,1))
x = np.hstack((np.hstack((const, z)), d))
y = 1 + -z + d + np.random.normal(scale = 0.5, size = (sample_size,1))

# make counter X
d1_counter = np.zeros((half_size,1))
d0_counter = np.ones((half_size,1))
d_counter = np.vstack((d0_counter, d1_counter))
x_counter = np.hstack((np.hstack((const, z)), d_counter))

### regression

In [5]:
# regression
regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(x, y)

# The coefficients
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [[ 0.99711691 -0.99813625  1.00582469]]


### neural nets

In [6]:
# neural nets
nn = MLPRegressor(hidden_layer_sizes=(10, ))
nn.fit(x, y)

/Users/keiikegami/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [19]:
# effect prediction on training sample
effec_on_nontreat = nn.predict(x_counter)[0:half_size] - nn.predict(x)[0:half_size]
effec_on_treat = nn.predict(x)[half_size:] - nn.predict(x_counter)[half_size:]

In [22]:
np.mean(effec_on_nontreat)

1.0076118531667566

In [23]:
np.mean(effec_on_treat)

1.0076121247729286

In [29]:
np.var(effec_on_nontreat)

0.0003308498320210379

In [30]:
np.var(effec_on_treat)

0.00032470210288134484

In [27]:
# effect on new samples
z_new = np.random.normal(size = (sample_size,1))

x_new1 = np.hstack((np.hstack((const, z_new)), np.ones((sample_size,1))))
x_new0 = np.hstack((np.hstack((const, z_new)), np.zeros((sample_size,1))))

effect_on_newsample = nn.predict(x_new1) - nn.predict(x_new0)

In [28]:
np.mean(effect_on_newsample)

1.0075487018840896

In [31]:
np.var(effect_on_newsample)

0.00033511288999682454

### specification 2

This specification include the cross terms in the true model. 